In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.ensemble import RandomForestClassifier
modelo_forest = joblib.load('modelo_random_forest.pkl')

In [2]:
# Lista com as 9 features finais utilizadas pelo modelo
FEATURES_FINAIS = [
    'ioi_36months', 'ioi_3months', 'valor_total_pedido', 'valor_quitado',
    'valor_vencido', 'quant_protestos', 'razao_vencido_total',
    'tem_restricoes', 'forma_pagamento_missing'
]

# Subconjunto: features numéricas
FEATURES_NUMERICAS = [
    'ioi_36months', 'ioi_3months', 'valor_total_pedido', 'valor_quitado',
    'valor_vencido', 'quant_protestos', 'razao_vencido_total'
]

# Subconjunto: features booleanas
FEATURES_BINARIAS = ['tem_restricoes', 'forma_pagamento_missing']


In [3]:
def criar_features_derivadas(dados):
    """
    - tem_restricoes: se o cliente tem protestos, ações ou dívidas
    - razao_vencido_total: proporção do valor vencido em relação ao total do pedido
    - forma_pagamento_missing: flag se a forma de pagamento está ausente
    """
    dados_processados = dados.copy()
    
    # Flag se existem restrições (protestos, ações judiciais ou dívidas vencidas)
    tem_protestos = dados.get('quant_protestos', 0) > 0
    tem_acoes = dados.get('quant_acao_judicial', 0) > 0
    tem_dividas = dados.get('dividas_vencidas_qtd', 0) > 0
    dados_processados['tem_restricoes'] = int(tem_protestos or tem_acoes or tem_dividas)
    
    # Razão valor vencido / total (evita divisão por zero somando +1)
    valor_vencido = dados.get('valor_vencido', 0)
    valor_total_pedido = dados.get('valor_total_pedido', 0)
    dados_processados['razao_vencido_total'] = valor_vencido / (valor_total_pedido + 1)
    
    # Variável indicando ausência de forma de pagamento
    forma_pagamento = dados.get('forma_pagamento')
    dados_processados['forma_pagamento_missing'] = int(forma_pagamento is None or pd.isna(forma_pagamento))
    
    return dados_processados


In [4]:
def preprocessar_para_modelo(dados):
    """
    - Cria features derivadas
    - Garante que todas as features finais estão presentes
    - Converte numéricas e binárias corretamente
    """
    dados_processados = criar_features_derivadas(dados)
    df_input = pd.DataFrame(index=[0])
    
    # Preenche todas as features necessárias
    for feature in FEATURES_FINAIS:
        df_input[feature] = dados_processados.get(feature, 0)
    
    # Converte numéricas
    for feature in FEATURES_NUMERICAS:
        df_input[feature] = pd.to_numeric(df_input[feature], errors='coerce').fillna(0)
    
    # Converte binárias
    for feature in FEATURES_BINARIAS:
        df_input[feature] = df_input[feature].astype(int)
    
    return df_input



In [5]:
def predict_default(input_dict):
    """
    Função de predição que recebe um dicionário com novos dados
    e retorna a predição de default
    """
    try:
        # Pré-processar dados
        df = preprocessar_para_modelo(input_dict)
        
        # Predição usando o modelo
        predicao = modelo_forest.predict(df)[0]
        
        return {"default": int(predicao)}
    
    except Exception as e:
        print(f"Erro na predição: {e}")
        return {"default": None}


In [7]:
input_dict = {"ioi_36months": 5,        
    "ioi_3months": 2,         
    "valor_total_pedido": 50000,
    "valor_quitado": 10000,
    "valor_vencido": 20000,
    "quant_protestos": 1,
    "quant_acao_judicial": 2,
    "dividas_vencidas_qtd": 3,
    "forma_pagamento": "boleto"}
resultado = predict_default(input_dict)
print(f"Resultado: {resultado}")



Resultado: {'default': 0}
